In [1]:
# warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'

ImportError: cannot import name 'get_openai_api_key' from 'utils' (/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/utils/__init__.py)